 ### Scipy minimization

In [1]:
import numpy as np
from scipy.optimize import minimize

$$\begin{array}{c} 
y = x^2 - 6x + 20 \\ 
minimize(y) \ and \ calculate\ \ x|_{y=min(y)}
\end{array}$$

In [2]:
def fun(x) :
    return x**2 - 6*x + 20

In [3]:
x0 = (3)
result = minimize(fun, x0 = x0, method = 'SLSQP')
x = result.x
x

array([3.])

In [4]:
fun(x)

array([11.])

 $$\begin{array}{c}
 minimize [ {(x_1 -1)}^2 + {(x_2 -2)}^2] \\  \\
 s.t. -x_1 + 3x_2 \leq 4 \ \ \ \ cons1 \\ \\
 x_1 + x_2 \leq 14 \ \ \ \ cons2 \\ \\
 x_1 -2x_2 \leq4 \ \ \ \ cons3 \\ \\
 0\leq x_1 , \ 0 \leq x_2 \\ \\
 x_0 = (2,0)
 \end{array}$$

In [5]:
fun2 = lambda x : (x[0]-1)**2 + (x[1]-2)**2
cons1 = lambda x : x[0]-3*x[1] + 4
cons2 = lambda x : -x[0]-x[1]+14
cons3 = lambda x : -x[0]+2*x[1]+4
cons = ({'type':'ineq','fun':cons1},
        {'type':'ineq','fun':cons2},
        {'type':'ineq','fun':cons3})
bnds = ((0,None),(0,None))
x0 = (2,0)
res = minimize(fun2, x0, method = 'SLSQP', bounds = bnds, constraints = cons)

In [6]:
res.x

array([1.1, 1.7])

 ### 문제 1. ★ portfolio volatility minimize
\begin{align}
\sigma_{PF}^2 = w \times \sum \times w^T \\
(\sum \ is \ cov \ matrix)
\end{align}

Covariance Matrix

| cov|A|B|C|
|------|---|---|---|
|A|Cov(A,A)|	Cov(A,B)|	Cov(A,C)|
|B|	Cov(B,A)|	Cov(B,B)|	Cov(B,C)|
|C|	Cov(C,A)|	Cov(C,B)|	Cov(C,C)|

주식 A,B,C의 변동성은 각각 20% ,30%, 50%이다.

주식 A,B,C의 상관계수행렬은 다음과 같다.

| corr|A|B|C|
|------|---|---|---|
|A|1|	0.6|	-0.2|
|B|0.6|	1|	0.0018|
|C|-0.2|	0.0018|	1|

포트폴리오 변동성을 최소화하기 위한 A,B,C의 최적 투자비중을 결정하시오.
$$
\begin{align}
minimize[\ \sigma_{PF}^2 = w \times \sum \times w^T \ ] \\
w_a + w_b + w_c = 1 \\
0\leq w_a \leq 1 \\
0\leq w_b \leq 1 \\
0\leq w_c \leq 1 \\
\end{align}
$$

방법 :

def port_variance(x, cov) :

    ## 이하생략 ##
    
    return float(variance)
    
cons_w = 함수 : 비율 array인 w를 넣었을 때 총합 -1

cons = ({'type':'eq','fun': cons_w})

bnds = 최솟값 0 , 최댓값 1 조건 3개

x0 = (0.3, 0.4, 0.3)

minimize(함수, x0, method = 'SLSQP', bounds = bnds , constraints = cons , arg = (cov))

In [7]:
def port_variance(x, cov) :
    port_var = x.reshape(1,-1).dot(cov).dot(x.reshape(-1,1))
    return float(port_var)

In [8]:
w = np.array([0.2, 0.3, 0.5]).reshape(1,-1)
vol = np.array([0.2, 0.15, 0.3]).reshape(1,-1)
corr = np.array([[1   , 0.6  , -0.2  ],                 
                 [0.6 ,   1  , 0.0018],
                 [-0.2,0.0018,1      ]])
cov = vol * vol.T * corr
port_variance(w,cov)

0.0259093

In [9]:
cons_w = lambda x : x.sum() -1
cons = ({'type':'eq','fun':cons_w})
bnds = ((0,1),(0,1),(0,1))
x0 = (0.3,0.4,0.3)
res = minimize(port_variance, x0, method = 'SLSQP', bounds = bnds, constraints = cons, args = (cov))

In [10]:
res.x

array([0.23071638, 0.5543315 , 0.21495212])

In [11]:
port_variance(res.x, cov)

0.016634708456504332

 ### 문제 2. A,B,C의 가정은 이전 문제와 같다고 가정하자.
 
 A,B,C의 각각 기대수익률은 10% 6% , 7%이다. 
 
 포트폴리오의 Sharp Ratio를 Maximize하기 위한 A,B,C 투자비중을 정하시오.
 
 (힌트) 먼저 포트폴리오의 샤프비율을 계산하는 함수를 만들고, 샤프 비율의 마이너스 값을 곱한것을 계산하는 함수를 만드시오.

In [12]:
expected_return = np.array([0.1,0.06,0.07])

In [13]:
def port_sharp(x,cov,expected_return) : 
    var = port_variance(x, cov)
    std = np.sqrt(var)
    port_ret = (x.reshape(-1) * expected_return).sum()
    return port_ret/std
def port_sharp_negative(x,cov,expected_return) :
    return -port_sharp(x,cov,expected_return)

In [14]:
port_sharp(w,cov,expected_return)

0.4535185132808075

In [15]:
port_sharp(res.x, cov ,expected_return)

0.5534238242902657

In [16]:
cons_w = lambda x : x.sum() -1
cons = ({'type':'eq','fun':cons_w})
bnds = ((0,1),(0,1),(0,1))
x0 = (0.3,0.4,0.3)
res = minimize(port_sharp_negative, x0, method = 'SLSQP', bounds = bnds, constraints = cons, args = (cov, expected_return))

In [17]:
res.x.round(3)

array([0.596, 0.144, 0.261])